In [1]:
from urllib.request import urlopen
from urllib.request import Request 
from urllib.parse import urlencode, quote_plus

import urllib.request as ul
import xmltodict
import json
import sys
import io
import pandas as pd
import requests

### 연월일 분리위한 데이터 프레임 만들기

In [2]:
start = pd.to_datetime('20130101')
end = pd.to_datetime('20190331') 
df = pd.DataFrame(columns=['date'])
df['date'] = pd.date_range(start,end,freq='D')

df['Year'] = df['date'].dt.year 
df['Month'] = df['date'].dt.month 
df['Day'] = df['date'].dt.day

df

,date,Year,Month,Day
0,2013-01-01,2013,1,1
1,2013-01-02,2013,1,2
2,2013-01-03,2013,1,3
3,2013-01-04,2013,1,4
4,2013-01-05,2013,1,5
...,...,...,...,...
2276,2019-03-27,2019,3,27
2277,2019-03-28,2019,3,28
2278,2019-03-29,2019,3,29
2279,2019-03-30,2019,3,30


In [3]:
### 월,일의 한자리(ex. 1,2,3...) 앞에 0붙여야함
def convert_month(df):
    for i in range(len(df)):
        month = df['Month'].iloc[i]
        if int(month) < 10:
            month = "0"+str(month)
            df['Month'].iloc[i] = month
            
def convert_day(df):
    for i in range(len(df)):
        day = df['Day'].iloc[i]
        if int(day) < 10:
            day = "0"+str(day)
            df['Day'].iloc[i] = day        

In [4]:
convert_month(df)
convert_day(df)
df

C:\Anaconda3\envs\chaewon\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,date,Year,Month,Day
0,2013-01-01,2013,01,01
1,2013-01-02,2013,01,02
2,2013-01-03,2013,01,03
3,2013-01-04,2013,01,04
4,2013-01-05,2013,01,05
...,...,...,...,...
2276,2019-03-27,2019,03,27
2277,2019-03-28,2019,03,28
2278,2019-03-29,2019,03,29
2279,2019-03-30,2019,03,30


### API로 음력 받아오기

In [5]:
data = pd.DataFrame()


for i in range(len(df)):
    
    Year = df['Year'].iloc[i]
    Month = df['Month'].iloc[i]
    Day = df['Day'].iloc[i]
               
    url = 'http://apis.data.go.kr/B090041/openapi/service/LrsrCldInfoService/getLunCalInfo?serviceKey=MS8w0NUzG28tbYJgzzGzNKYispIbGKI3zFOsb8cV08xuh%2FCqBHlYn%2FfQcLySJxjsN0dvB93Ol%2BVQKftGRQG9ew%3D%3D&solYear='+str(Year)+'&solMonth='+str(Month)+'&solDay='+str(Day)

    requestd = Request(url)
    requestd.get_method = lambda: 'GET'
    response_body = urlopen(requestd).read()
    resp = xmltodict.parse(response_body)
    resp_dic = json.loads(json.dumps(resp))
    #print(resp_dic)
    #print(i)

    data = data.append(resp_dic['response']['body']['items']['item'],ignore_index=True)

print("********end********")

********end********


In [6]:
raw_api = data[['lunYear','lunMonth','lunDay']]
raw_api

,lunYear,lunMonth,lunDay
0,2012,11,20
1,2012,11,21
2,2012,11,22
3,2012,11,23
4,2012,11,24
...,...,...,...
2276,2019,02,21
2277,2019,02,22
2278,2019,02,23
2279,2019,02,24


### 만들어둔 데이터 프레임이랑 합치기

In [7]:
lun_2013 = pd.concat([df,raw_api],axis=1) 
lun_2013

,date,Year,Month,Day,lunYear,lunMonth,lunDay
0,2013-01-01,2013,01,01,2012,11,20
1,2013-01-02,2013,01,02,2012,11,21
2,2013-01-03,2013,01,03,2012,11,22
3,2013-01-04,2013,01,04,2012,11,23
4,2013-01-05,2013,01,05,2012,11,24
...,...,...,...,...,...,...,...
2276,2019-03-27,2019,03,27,2019,02,21
2277,2019-03-28,2019,03,28,2019,02,22
2278,2019-03-29,2019,03,29,2019,02,23
2279,2019-03-30,2019,03,30,2019,02,24


In [8]:
lun_2013.to_csv("./make_data/0B_음력날짜.csv",index=False) ##여기까지 20130101~20190331

### 데이터 프레임 만들기 (19881001~20121231)

In [9]:
start = pd.to_datetime('19881001')
end = pd.to_datetime('20121231') 
df2 = pd.DataFrame(columns=['date'])
df2['date'] = pd.date_range(start,end,freq='D')

df2['Year'] = df2['date'].dt.year 
df2['Month'] = df2['date'].dt.month 
df2['Day'] = df2['date'].dt.day

df2

,date,Year,Month,Day
0,1988-10-01,1988,10,1
1,1988-10-02,1988,10,2
2,1988-10-03,1988,10,3
3,1988-10-04,1988,10,4
4,1988-10-05,1988,10,5
...,...,...,...,...
8853,2012-12-27,2012,12,27
8854,2012-12-28,2012,12,28
8855,2012-12-29,2012,12,29
8856,2012-12-30,2012,12,30


In [10]:
convert_month(df2)
convert_day(df2)
df2

C:\Anaconda3\envs\chaewon\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,date,Year,Month,Day
0,1988-10-01,1988,10,01
1,1988-10-02,1988,10,02
2,1988-10-03,1988,10,03
3,1988-10-04,1988,10,04
4,1988-10-05,1988,10,05
...,...,...,...,...
8853,2012-12-27,2012,12,27
8854,2012-12-28,2012,12,28
8855,2012-12-29,2012,12,29
8856,2012-12-30,2012,12,30


### API 받아오기 (19881001~20121231)

In [11]:
data = pd.DataFrame()


for i in range(len(df2)):
    
    Year = df2['Year'].iloc[i]
    Month = df2['Month'].iloc[i]
    Day = df2['Day'].iloc[i]
               
    url = 'http://apis.data.go.kr/B090041/openapi/service/LrsrCldInfoService/getLunCalInfo?serviceKey=MS8w0NUzG28tbYJgzzGzNKYispIbGKI3zFOsb8cV08xuh%2FCqBHlYn%2FfQcLySJxjsN0dvB93Ol%2BVQKftGRQG9ew%3D%3D&solYear='+str(Year)+'&solMonth='+str(Month)+'&solDay='+str(Day)

    requestd = Request(url)
    requestd.get_method = lambda: 'GET'
    response_body = urlopen(requestd).read()
    resp = xmltodict.parse(response_body)
    resp_dic = json.loads(json.dumps(resp))
    #print(resp_dic)
    #print(i)

    data = data.append(resp_dic['response']['body']['items']['item'],ignore_index=True)

print("********end********")

********end********


In [12]:
raw_api2 = data[['lunYear','lunMonth','lunDay']]

In [13]:
lun_1988 = pd.concat([df2,raw_api2],axis=1) #열방향으로 음력합침
lun_1988

,date,Year,Month,Day,lunYear,lunMonth,lunDay
0,1988-10-01,1988,10,01,1988,08,21
1,1988-10-02,1988,10,02,1988,08,22
2,1988-10-03,1988,10,03,1988,08,23
3,1988-10-04,1988,10,04,1988,08,24
4,1988-10-05,1988,10,05,1988,08,25
...,...,...,...,...,...,...,...
8853,2012-12-27,2012,12,27,2012,11,15
8854,2012-12-28,2012,12,28,2012,11,16
8855,2012-12-29,2012,12,29,2012,11,17
8856,2012-12-30,2012,12,30,2012,11,18


In [14]:
lun_final_1988 = pd.concat([lun_1988,lun_2013],axis=0) #행방향으로 날짜합침

In [15]:
lun_final_1988

,date,Year,Month,Day,lunYear,lunMonth,lunDay
0,1988-10-01,1988,10,01,1988,08,21
1,1988-10-02,1988,10,02,1988,08,22
2,1988-10-03,1988,10,03,1988,08,23
3,1988-10-04,1988,10,04,1988,08,24
4,1988-10-05,1988,10,05,1988,08,25
...,...,...,...,...,...,...,...
2276,2019-03-27,2019,03,27,2019,02,21
2277,2019-03-28,2019,03,28,2019,02,22
2278,2019-03-29,2019,03,29,2019,02,23
2279,2019-03-30,2019,03,30,2019,02,24


In [17]:
lun_final_1988.reset_index(drop=True, inplace=True)
lun_final_1988

,date,Year,Month,Day,lunYear,lunMonth,lunDay
0,1988-10-01,1988,10,01,1988,08,21
1,1988-10-02,1988,10,02,1988,08,22
2,1988-10-03,1988,10,03,1988,08,23
3,1988-10-04,1988,10,04,1988,08,24
4,1988-10-05,1988,10,05,1988,08,25
...,...,...,...,...,...,...,...
11134,2019-03-27,2019,03,27,2019,02,21
11135,2019-03-28,2019,03,28,2019,02,22
11136,2019-03-29,2019,03,29,2019,02,23
11137,2019-03-30,2019,03,30,2019,02,24


In [18]:
lun_final_1988.to_csv("./make_data/0B_1988_음력날짜.csv",index=False)